In [32]:
# Importing all the REQUIRED packages 
import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML

In [33]:
sm=pd.read_csv('data.csv')

In [34]:
display(HTML(sm.head(10).to_html()))

In [35]:
sm.head()
sm= sm.drop('cta_sites',1)
sm= sm.drop('incident_date',1)
sm= sm.drop('incident_time',1)
sm= sm.drop('time_category',1)
sm= sm.drop('incident_type',1)
sm= sm.drop('moonrise',1)
sm= sm.drop('moonset',1)
sm= sm.drop('weather_description',1)
sm= sm.drop('week_day',1)

cols=sm.columns


# UNDERSTANDING THE DATA

CTA SITE HAS UNIQUE CODE FOR EACH STATION

WEEKCODE -
FRIDAY:1
WEDNESDAY:2
MONDAY:6
THURDAY:3
Saturday:4
Sunday:5
Tuesday:7

ROUNDING OFF incident_time from 3.37 to 3

TIME_CATEGORY -
late_night:1
night:2
early_morning:3
morning:5
evening:4
afternoon:6

WEATHER_CODE has 30 unique nominal values so converted to unique numericcal values

In [36]:
display(HTML(sm.head(10).to_html()))

In [37]:
sm= sm.drop('sunrise',1)
sm= sm.drop('sunset',1)

In [38]:
sm.head()

site_id  week_code  weekend  holiday  incident_time.1  time_code  atm  \
0       14          1        0        0                3          1    0   
1        5          2        0        1                2          1    0   
2       12          2        0        1                3          1    0   
3      143          3        0        0                3          1    0   
4       15          3        0        0               22          2    1   

   restaurant  avgtemp  maxtemp  totalsnow  sunhour  Temperature  Humidity  \
0           0       32       34       0.12      5.0           32        76   
1           0       34       39       0.00      8.7           28        82   
2           0       34       39       0.00      8.7           28        83   
3           0       41       45       0.00      5.2           43        76   
4           1       41       45       0.00      5.2           43        90   

   cloudcover  heatindex  hours_after_sunrise  hours_to_sunset  incident  \
0         100         32                -4.45            14.45         1   
1          30         28                -6.30            15.50         0   
2          28         28                -5.30            14.50         0   
3          74         43                -5.30            14.52         1   
4          64         43                13.70            -4.48         0   

   weather_code  
0            76  
1            82  
2            82  
3            82  
4            82

In [39]:
# Pre Processing the Data
print('Column Datatypes:\n',sm.dtypes)
# Converting Nominal Attributes to Binary Values
sm_raw=sm.copy(deep=True) 
sm_knn=sm.copy(deep=True)

Column Datatypes:
 site_id                  int64
week_code                int64
weekend                  int64
holiday                  int64
incident_time.1          int64
time_code                int64
atm                      int64
restaurant               int64
avgtemp                  int64
maxtemp                  int64
totalsnow              float64
sunhour                float64
Temperature              int64
Humidity                 int64
cloudcover               int64
heatindex                int64
hours_after_sunrise    float64
hours_to_sunset        float64
incident                 int64
weather_code             int64
dtype: object


In [40]:
# Normalizing 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
cols_numeric = sm_knn.select_dtypes(include=numerics).columns.tolist()
print('Selected numerical columns:\n',cols_numeric) 

Selected numerical columns:
 ['site_id', 'week_code', 'weekend', 'holiday', 'incident_time.1', 'time_code', 'atm', 'restaurant', 'avgtemp', 'maxtemp', 'totalsnow', 'sunhour', 'Temperature', 'Humidity', 'cloudcover', 'heatindex', 'hours_after_sunrise', 'hours_to_sunset', 'incident', 'weather_code']


In [41]:
#Using Min-Max Normalization to scale [0, 1]
for col in cols_numeric:
    sm_knn[col]=(sm_knn[col]-sm_knn[col].min())/(sm_knn[col].max()-sm_knn[col].min())

In [42]:
display(HTML(sm_knn.head(10).to_html()))

# Build KNN Model

In [43]:
# Build KNN Model
# N-fold cross Validation 

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Building and Evaluating Models
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# KNN requires Label so we need to Pre-Process the Label values

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score


y = sm_knn['incident'] # Defining label as nominal values
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y) 
sm_knn['incident'] = y_encoded
x = sm_knn.drop('incident',1)
y = sm_knn['incident'] 




for k in range(1, 20, 2): 
    clf=neighbors.KNeighborsClassifier(k, weights='uniform')
    acc=cross_val_score(clf, x, y, cv=5, scoring='accuracy').mean()
    print('K =', k, ', Accuracy: ',acc)
 

K = 1 , Accuracy:  0.6035974720466699
K = 3 , Accuracy:  0.6168761719563859
K = 5 , Accuracy:  0.645003125217029
K = 7 , Accuracy:  0.6499062434891312
K = 9 , Accuracy:  0.6664907285228142
K = 11 , Accuracy:  0.6581151468852003
K = 13 , Accuracy:  0.673102298770748
K = 15 , Accuracy:  0.6648378359608306
K = 17 , Accuracy:  0.6880894506562956
K = 19 , Accuracy:  0.686436558094312


In [46]:
sm_raw.head()

site_id  week_code  weekend  holiday  incident_time.1  time_code  atm  \
0       14          1        0        0                3          1    0   
1        5          2        0        1                2          1    0   
2       12          2        0        1                3          1    0   
3      143          3        0        0                3          1    0   
4       15          3        0        0               22          2    1   

   restaurant  avgtemp  maxtemp  totalsnow  sunhour  Temperature  Humidity  \
0           0       32       34       0.12      5.0           32        76   
1           0       34       39       0.00      8.7           28        82   
2           0       34       39       0.00      8.7           28        83   
3           0       41       45       0.00      5.2           43        76   
4           1       41       45       0.00      5.2           43        90   

   cloudcover  heatindex  hours_after_sunrise  hours_to_sunset  incident  \
0         100         32                -4.45            14.45         1   
1          30         28                -6.30            15.50         0   
2          28         28                -5.30            14.50         0   
3          74         43                -5.30            14.52         1   
4          64         43                13.70            -4.48         0   

   weather_code  
0            76  
1            82  
2            82  
3            82  
4            82

# Naive Bayes

In [49]:
# Requirements: 1)Nominal features 2)Assumption of conditionally indepenence
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

In [50]:
# Pre-processing the data
print('Column data types:\n',sm_raw.dtypes)
sm_nb=sm_raw.copy(deep=True)


Column data types:
 site_id                  int64
week_code                int64
weekend                  int64
holiday                  int64
incident_time.1          int64
time_code                int64
atm                      int64
restaurant               int64
avgtemp                  int64
maxtemp                  int64
totalsnow              float64
sunhour                float64
Temperature              int64
Humidity                 int64
cloudcover               int64
heatindex                int64
hours_after_sunrise    float64
hours_to_sunset        float64
incident                 int64
weather_code             int64
dtype: object


In [51]:
# Convert Numerical to Categorical Data
sm_nb['site_id'] = pd.cut(sm_nb['site_id'],2)
sm_nb['week_code'] = pd.cut(sm_nb['week_code'],3)
sm_nb['weekend'] = pd.cut(sm_nb['weekend'],2)
sm_nb['holiday'] = pd.cut(sm_nb['holiday'],2)
sm_nb['incident_time.1'] = pd.cut(sm_nb['incident_time.1'],3)
sm_nb['time_code'] = pd.cut(sm_nb['time_code'],2)
sm_nb['atm'] = pd.cut(sm_nb['atm'],3)
sm_nb['restaurant'] = pd.cut(sm_nb['restaurant'],3)
sm_nb['avgtemp'] = pd.cut(sm_nb['avgtemp'],2)
sm_nb['maxtemp'] = pd.cut(sm_nb['maxtemp'],2)
sm_nb['totalsnow'] = pd.cut(sm_nb['totalsnow'],2)
sm_nb['sunhour'] = pd.cut(sm_nb['sunhour'],2)
sm_nb['Temperature'] = pd.cut(sm_nb['Temperature'],3)
sm_nb['Humidity'] = pd.cut(sm_nb['Humidity'],2)
sm_nb['cloudcover'] = pd.cut(sm_nb['cloudcover'],2)
sm_nb['heatindex'] = pd.cut(sm_nb['heatindex'],2)
sm_nb['hours_after_sunrise'] = pd.cut(sm_nb['hours_after_sunrise'],3)
sm_nb['hours_to_sunset'] = pd.cut(sm_nb['hours_to_sunset'],3)
sm_nb['weather_code'] = pd.cut(sm_nb['weather_code'],3)
sm_nb['incident'] = pd.cut(sm_nb['incident'],2)


In [53]:
sm_nb.dtypes

site_id                category
week_code              category
weekend                category
holiday                category
incident_time.1        category
time_code              category
atm                    category
restaurant             category
avgtemp                category
maxtemp                category
totalsnow              category
sunhour                category
Temperature            category
Humidity               category
cloudcover             category
heatindex              category
hours_after_sunrise    category
hours_to_sunset        category
incident               category
weather_code           category
dtype: object

In [56]:
y=sm_nb['incident']
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

display('Data Example',HTML(sm_nb.head(5).to_html()))

print(sm_nb.dtypes)

# Transform Categorical Data to Numerical Data

print(sm_nb.dtypes)
sm_nb=pd.get_dummies(sm_nb.drop('incident',axis=1))
sm_nb['incident']=y_encoded
display(HTML(sm_nb.head(5).to_html()))

#N-Fold Cross Validation 5-cross
y=sm_nb['incident']
x=sm_nb.drop('incident',axis=1)
clf = GaussianNB()
acc=cross_val_score(clf, x, y, cv=5, scoring='accuracy').mean()
print("Accuracy by N-fold Cross Validation:",acc)


'Data Example'

site_id                category
week_code              category
weekend                category
holiday                category
incident_time.1        category
time_code              category
atm                    category
restaurant             category
avgtemp                category
maxtemp                category
totalsnow              category
sunhour                category
Temperature            category
Humidity               category
cloudcover             category
heatindex              category
hours_after_sunrise    category
hours_to_sunset        category
incident               category
weather_code           category
dtype: object
site_id                category
week_code              category
weekend                category
holiday                category
incident_time.1        category
time_code              category
atm                    category
restaurant             category
avgtemp                category
maxtemp                category
totalsnow              cat

Accuracy by N-fold Cross Validation: 0.5472324466976872
